# LUME-services demo
In this notebook, we will configure LUME-services to use the service configuration used to launch our docker-compose services. Make sure you've completed all steps outlined in https://slaclab.github.io/lume-services/demo/.

In [1]:
import logging
logging.basicConfig(level=logging.INFO)  # Lets check the logs

## Configure services
LUME-services is packages with a configuration utility that reads environment variables and initializes services:

In [2]:
from lume_services import config
config.configure()

INFO:lume_services.config:Configuring LUME-services environment...
INFO:lume_services.config:Environment configured.


## if you're running this many time, creation will fail because of uniqueness... You can reset since this is a dev server

In [3]:
#model_db_service._reset()

## Create a model
The LUME-serives Model object handles all model operations.

In [4]:
from lume_services.models import Model

model = Model.create_model(
    author = "Jackie Garrahan",
    laboratory = "slac",
    facility = "lcls",
    beampath = "cu_hxr",
    description = "test_model"
)
model

INFO:lume_services.services.models.db.db:ModelDB inserting: INSERT INTO model (author, laboratory, facility, beampath, description) VALUES (:author, :laboratory, :facility, :beampath, :description)
INFO:lume_services.services.models.db.db:Sucessfully executed: INSERT INTO model (author, laboratory, facility, beampath, description) VALUES (:author, :laboratory, :facility, :beampath, :description)
INFO:lume_services.services.models.db.db:ModelDB selecting: SELECT model.model_id, model.created, model.author, model.laboratory, model.facility, model.beampath, model.description 
FROM model 
WHERE model.model_id = :model_id_1


Model(metadata=Model(                     model_id=1,                     created=datetime.datetime(2022, 9, 9, 21, 29, 32),                     author='Jackie Garrahan'),                     laboratory='slac',                     facility='lcls',                     beampath='cu_hxr',                     description='test_model'                 ), deployment=None, results=None)

## Create a project
Workflows are organized by the Prefect scheduler into different projects. Below, we access the configured services directly (TODO create project registry utility)

In [5]:
# create a project
model_db_service = config.context.model_db_service()
scheduling_service = config.context.scheduling_service()

project_name = model_db_service.store_project(
    project_name="test", description="my_description"
)
scheduling_service.create_project("test")

INFO:lume_services.services.models.db.db:ModelDB inserting: INSERT INTO project (project_name, description) VALUES (:project_name, :description)
INFO:lume_services.services.models.db.db:Sucessfully executed: INSERT INTO project (project_name, description) VALUES (:project_name, :description)


You can now find this project in you Prefect UI at http://localhost:8080


![project](../files/project_nav.png)

## Create a deployment for your model

In [7]:
source_path = "https://github.com/jacquelinegarrahan/my-model/releases/download/v0.0.9/my_model-0.0.9.tar.gz"

# populates local channel
model.store_deployment(source_path, project_name="test")

INFO:lume_services.environment.solver:https://github.com/jacquelinegarrahan/my-model/releases/download/v0.0.9/my_model-0.0.9.tar.gz saved to /tmp/lume-services/my_model-0.0.9.tar.gz
INFO:lume_services.environment.solver:https://github.com/jacquelinegarrahan/my-model/releases/download/v0.0.9/my_model-0.0.9.tar.gz saved to /tmp/lume-services/my_model-0.0.9.tar.gz
INFO:lume_services.environment.solver:Using conda channels conda-forge to install dependencies.
INFO:conda.core.link:initializing UnlinkLinkTransaction with
  target_prefix: /tmp/lume-services/tmp_env
  unlink_precs:
    
  link_precs:
    

INFO:conda.notices.http:Received 404 when trying to GET https://repo.anaconda.com/pkgs/r/notices.json
INFO:conda.notices.http:Received 404 when trying to GET https://repo.anaconda.com/pkgs/main/notices.json
INFO:conda.notices.http:Received 302 when trying to GET https://conda.anaconda.org/conda-forge/notices.json
INFO:lume_services.environment.solver:Resolving conda dependencies...
INFO:lume

INFO:lume_services.environment.solver:Collecting /Users/jacquelinegarrahan/sandbox/lume-services/local-conda-channel/linux-64/libbrotlidec-1.0.9-h166bdaf_7.tar.bz2
INFO:lume_services.environment.solver:Collecting /Users/jacquelinegarrahan/sandbox/lume-services/local-conda-channel/linux-64/libblas-3.9.0-16_linux64_openblas.tar.bz2
INFO:lume_services.environment.solver:Collecting /Users/jacquelinegarrahan/sandbox/lume-services/local-conda-channel/linux-64/reproc-cpp-14.2.3-h9c3ff4c_0.tar.bz2
INFO:lume_services.environment.solver:Collecting /Users/jacquelinegarrahan/sandbox/lume-services/local-conda-channel/linux-64/libedit-3.1.20191231-he28a2e2_2.tar.bz2
INFO:lume_services.environment.solver:Collecting /Users/jacquelinegarrahan/sandbox/lume-services/local-conda-channel/linux-64/readline-8.1.2-h0f457ee_0.tar.bz2
INFO:lume_services.environment.solver:Collecting /Users/jacquelinegarrahan/sandbox/lume-services/local-conda-channel/linux-64/zstd-1.5.2-h6239696_4.tar.bz2
INFO:lume_services.envi

INFO:lume_services.environment.solver:Collecting /Users/jacquelinegarrahan/sandbox/lume-services/local-conda-channel/noarch/pytzdata-2020.1-pyh9f0ad1d_0.tar.bz2
INFO:lume_services.environment.solver:Collecting /Users/jacquelinegarrahan/sandbox/lume-services/local-conda-channel/noarch/typing_extensions-4.3.0-pyha770c72_0.tar.bz2
INFO:lume_services.environment.solver:Collecting /Users/jacquelinegarrahan/sandbox/lume-services/local-conda-channel/noarch/natsort-8.2.0-pyhd8ed1ab_0.tar.bz2
INFO:lume_services.environment.solver:Collecting /Users/jacquelinegarrahan/sandbox/lume-services/local-conda-channel/noarch/fsspec-2022.8.2-pyhd8ed1ab_0.tar.bz2
INFO:lume_services.environment.solver:Collecting /Users/jacquelinegarrahan/sandbox/lume-services/local-conda-channel/noarch/tblib-1.7.0-pyhd8ed1ab_0.tar.bz2
INFO:lume_services.environment.solver:Collecting /Users/jacquelinegarrahan/sandbox/lume-services/local-conda-channel/noarch/locket-1.0.0-pyhd8ed1ab_0.tar.bz2
INFO:lume_services.environment.solv

INFO:lume_services.environment.solver:Collecting /Users/jacquelinegarrahan/sandbox/lume-services/local-conda-channel/linux-64/libmambapy-0.25.0-py39hd55135b_2.tar.bz2
INFO:lume_services.environment.solver:Collecting /Users/jacquelinegarrahan/sandbox/lume-services/local-conda-channel/linux-64/dependency_injector-4.40.0-py39hb9d737c_0.tar.bz2
INFO:lume_services.environment.solver:Collecting /Users/jacquelinegarrahan/sandbox/lume-services/local-conda-channel/linux-64/numpy-1.23.2-py39hba7629e_0.tar.bz2
INFO:lume_services.environment.solver:Collecting /Users/jacquelinegarrahan/sandbox/lume-services/local-conda-channel/linux-64/click-8.1.3-py39hf3d152e_0.tar.bz2
INFO:lume_services.environment.solver:Collecting /Users/jacquelinegarrahan/sandbox/lume-services/local-conda-channel/linux-64/pyyaml-6.0-py39hb9d737c_4.tar.bz2
INFO:lume_services.environment.solver:Collecting /Users/jacquelinegarrahan/sandbox/lume-services/local-conda-channel/linux-64/pydantic-1.10.2-py39hb9d737c_0.tar.bz2
INFO:lume

INFO:lume_services.environment.solver:Dependency installation complete
  Running command python setup.py egg_info
  running egg_info
  creating /private/var/folders/nh/g2v_nmtj7t1g94gmjtgjrk3r0000gn/T/pip-pip-egg-info-mlmz7wlt/my_model.egg-info
  writing /private/var/folders/nh/g2v_nmtj7t1g94gmjtgjrk3r0000gn/T/pip-pip-egg-info-mlmz7wlt/my_model.egg-info/PKG-INFO
  writing dependency_links to /private/var/folders/nh/g2v_nmtj7t1g94gmjtgjrk3r0000gn/T/pip-pip-egg-info-mlmz7wlt/my_model.egg-info/dependency_links.txt
  writing entry points to /private/var/folders/nh/g2v_nmtj7t1g94gmjtgjrk3r0000gn/T/pip-pip-egg-info-mlmz7wlt/my_model.egg-info/entry_points.txt
  writing requirements to /private/var/folders/nh/g2v_nmtj7t1g94gmjtgjrk3r0000gn/T/pip-pip-egg-info-mlmz7wlt/my_model.egg-info/requires.txt
  writing top-level names to /private/var/folders/nh/g2v_nmtj7t1g94gmjtgjrk3r0000gn/T/pip-pip-egg-info-mlmz7wlt/my_model.egg-info/top_level.txt
  writing manifest file '/private/var/folders/nh/g2v_nm

IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry 'da10d0a5ee39fb2b45d4da5c3d972141d139f876370871b03dde90772e44d764' for key 'deployment._sha256_unique'")
[SQL: INSERT INTO deployment (version, package_import_name, asset_dir, source, sha256, image, is_live, model_id) VALUES (%(version)s, %(package_import_name)s, %(asset_dir)s, %(source)s, %(sha256)s, %(image)s, %(is_live)s, %(model_id)s)]
[parameters: {'version': '0.0.9', 'package_import_name': 'my_model', 'asset_dir': None, 'source': 'https://github.com/jacquelinegarrahan/my-model/releases/download/v0.0.9/my_model-0.0.9.tar.gz', 'sha256': 'da10d0a5ee39fb2b45d4da5c3d972141d139f876370871b03dde90772e44d764', 'image': 'jgarrahan/lume-services-prefect:latest', 'is_live': 1, 'model_id': 1}]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

## Run the Prefect workflow directly

In [9]:
model.deployment.flow.prefect_flow.run(**{
                        "input1": 1, 
                        "input2": 2, 
                        "filename": "/Users/jacquelinegarrahan/sandbox/lume-services/test_file.txt", 
                        "filesystem_identifier":"local"}
)

[2022-09-09 14:36:04-0700] INFO - prefect.FlowRunner | Beginning Flow run for 'my-model'


INFO:prefect.FlowRunner:Beginning Flow run for 'my-model'


[2022-09-09 14:36:04-0700] INFO - prefect.TaskRunner | Task 'input2': Starting task run...


INFO:prefect.TaskRunner:Task 'input2': Starting task run...


[2022-09-09 14:36:04-0700] INFO - prefect.TaskRunner | Task 'input2': Finished task run for task with final state: 'Success'


...

[2022-09-09 14:36:05-0700] INFO - prefect.TaskRunner | Task 'save_file': Finished task run for task with final state: 'Success'


INFO:prefect.TaskRunner:Task 'save_file': Finished task run for task with final state: 'Success'


[2022-09-09 14:36:05-0700] INFO - prefect.FlowRunner | Flow run FAILED: some reference tasks failed.


INFO:prefect.FlowRunner:Flow run FAILED: some reference tasks failed.


<Failed: "Some reference tasks failed.">

In [12]:
model_db_service.get_flow(deployment_id=)

Deployment(metadata=None, project=Project(metadata=Project(                 project_name='test',                 description='my_description',                 )), dependencies=[Dependencies(                     id=1,                     name='_libgcc_mutex',                     source='https://conda.anaconda.org/t/jr-ac5b1b42-c470-4696-a06b-9e98c0d40429/conda-forge/linux-64/_libgcc_mutex-0.1-conda_forge.tar.bz2',                     local_source='/Users/jacquelinegarrahan/sandbox/lume-services/local-conda-channel/linux-64/_libgcc_mutex-0.1-conda_forge.tar.bz2'                     version='0.1'                 ), Dependencies(                     id=2,                     name='libstdcxx-ng',                     source='https://conda.anaconda.org/t/jr-ac5b1b42-c470-4696-a06b-9e98c0d40429/conda-forge/linux-64/libstdcxx-ng-12.1.0-ha89aaad_16.tar.bz2',                     local_source='/Users/jacquelinegarrahan/sandbox/lume-services/local-conda-channel/linux-64/libstdcxx-ng-12.1.0-ha89aaad

## Run the workflow inside the service cluster
We can use the model interface to directly deploy workflows. When sourcing our environment (`docs/examples/demo.env`), we defined a mount point for the file system using the alias `/lume-services/data`. Let's kick off this workflow and save the file output to that directory: 

In [10]:
model.run_and_return(
    parameters={
        "input1": "hey", 
        "input2": "jackie", 
        "filename": "/lume-services/data/test_file.txt", 
        "filesystem_identifier":"mounted"}
)

INFO:lume_services.services.scheduling.backends.server:Creating Prefect flow run for None with parameters {'input1': 'hey', 'input2': 'jackie', 'filename': '/lume-services/data/test_file.txt', 'filesystem_identifier': 'mounted'} and run_config {"labels": ["lume-services"], "env": {"EXTRA_CONDA_PACKAGES": "_libgcc_mutex libstdcxx-ng ld_impl_linux-64 ca-certificates libgfortran5 libgomp libgfortran-ng _openmp_mutex libgcc-ng lzo xorg-libxdmcp xorg-libxau pthread-stubs libuuid libdeflate lerc jpeg libbrotlicommon libev c-ares libwebp-base ripgrep liblief patchelf patch lz4-c icu libopenblas reproc libiconv keyutils yaml-cpp xz openssl ncurses libzlib libffi bzip2 yaml libnsl libxcb libbrotlienc libbrotlidec libblas reproc-cpp libedit readline zstd libpng libxml2 libssh2 libnghttp2 tk libsqlite libsolv brotli-bin libcblas liblapack krb5 libtiff freetype libarchive sqlite brotli libcurl openjpeg lcms2 hdf5 libmamba tzdata pybind11-abi python wheel setuptools pip munkres cycler soupsieve cac

ValueError: One of flow_id or version_group_id must be provided

In [ ]:
model.results()

In [ ]:
from lume_services.models import Model

model = Model(model_id=1)

In [ ]:
model.load_deployment()

In [ ]:
model.run_and_return(parameters={
                        "input1": "hey", 
                        "input2": "jackie", 
                        "filename": f"/Users/jgarra/sandbox/lume-services/test_file.txt", 
                        "filesystem_identifier":"local"
        }
)